# Creates EC lists and EC dictionaries from raw JGI data

In [1]:
import numpy as np
import pandas as pd
import glob
import os
import json
from builtins import any

In [2]:
### location of your data goes here
paths = ['/Users/dgagler/School/SecondaryProject/RedoxEnzymes/JGI_2019/Eukarya/taxon_ids_all/taxon_ids',
         '/Users/dgagler/School/SecondaryProject/RedoxEnzymes/JGI_2019/Archaea/taxon_ids_all/taxon_ids',
         '/Users/dgagler/School/SecondaryProject/RedoxEnzymes/JGI_2019/Metagenomes/taxon_ids',
        '/Users/dgagler/School/SecondaryProject/RedoxEnzymes/JGI_2019/Bacteria/taxon_ids']

files = {}

for path in paths:
    files[path] = glob.glob(os.path.join(path, '*.json'))
    
eukarya_raw = files[paths[0]]
archaea_raw = files[paths[1]]
metagenomes_raw = files[paths[2]]
bacteria_raw = files[paths[3]]

### folders to dump ec lists to...make them ahead of time!
eukarya_dump_path = '/Users/dgagler/School/SecondaryProject/RedoxEnzymes/JGI_2019/Eukarya/ec_lists'
archaea_dump_path = '/Users/dgagler/School/SecondaryProject/RedoxEnzymes/JGI_2019/Archaea/ec_lists'
metagenome_dump_path = '/Users/dgagler/School/SecondaryProject/RedoxEnzymes/JGI_2019/Metagenomes/ec_lists'
bacteria_dump_path = '/Users/dgagler/School/SecondaryProject/RedoxEnzymes/JGI_2019/Bacteria/ec_lists'

In [3]:
def create_eclists_ecdict_genome(raw_data, dump_path):
    
    ec_dict = {}
    num_ec_dict = {}
    
    for i in raw_data:
        with open(i, 'r') as f:
            data = json.load(f)
            filename = '{}.json'.format(data['metadata']['Taxon ID'])
            taxon_id = data['metadata']['Taxon ID']
            
            if len(data) == 2: ### this part is necessary because some of the .jsons have no enzymes
                enzymes = (list(data['enzymes'].keys())) ### if the data is a dict of metadata and enzymes, proceed
                enzymes = [e[3:] for e in enzymes]
                num_ecs = len(enzymes)
                
            num_ec_dict[taxon_id] = num_ecs
            ec_dict[taxon_id] = enzymes    
            
            with open(dump_path + '/{}'.format(filename), 'w') as f:
                json.dump(enzymes, f)
    with open(dump_path + '/ecdict.json', 'w') as f:
        json.dump(ec_dict, f)
    with open(dump_path + '/num_ecs.json', 'w') as f:
        json.dump(num_ec_dict, f)

- The structure of json dicts for metagenomes are different than other genomes. Instead of having 2 dicts, dict['metadata'] and dict['enzymes'], they have dict['metadata'], dict['assembled'], dict['unassembled'], and dict['both']

In [4]:
### structure of json dicts for metagenomes are different than other genomes
### instead of having 2 dicts, dict['metadata'] and dict['enzymes'],
### they have dict['metadata'], dict['assembled'], dict['unassembled'], and dict['both']

def create_eclists_ecdict_metagenome(raw_data, dump_path):
    ec_dict = {}
    num_ec_dict = {}
    
    for i in raw_data:
        with open(i, 'r') as f:
            data = json.load(f)
            filename = '{}.json'.format(data['metadata']['Taxon Object ID'])
            taxon_id = data['metadata']['Taxon Object ID']
            
            if 'assembled' in data: 
                enzymes = data['assembled'] ### if the data is a dict of metadata and enzymes, proceed
            enzymes = [e[3:] for e in enzymes]
            num_ecs = len(enzymes)
            
            ec_dict[taxon_id] = enzymes
            num_ec_dict[taxon_id] = num_ecs
            
            
            with open(dump_path + '/{}'.format(filename), 'w') as f:
                json.dump(enzymes, f)
    with open(dump_path + '/ecdict.json', 'w') as f:
        json.dump(ec_dict, f)
    with open(dump_path + '/num_ecs.json', 'w') as f:
        json.dump(num_ec_dict, f)

In [5]:
### get only metagenomes with oxygen concentrations...make dict of their metadata and enzyems

pilar_dump_path = '/Users/dgagler/School/SecondaryProject/RedoxEnzymes/JGI_2019/Metagenomes/pilar'

for i in metagenomes_raw:
    teste_dict = {}
    with open(i, 'r') as f:
        teste = json.load(f)
        taxon_id = teste['metadata']['Taxon Object ID']
        filename = '{}.json'.format(taxon_id)
        for k,v in teste['metadata'].items():
            if 'Oxygen' in k:
                #teste_dict[taxon_id] = teste['metadata']
                if 'assembled' in teste: 
                    metadata = teste['metadata']
                    enzymes = teste['assembled'].keys() ### if the data is a dict of metadata and enzymes, proceed
                    enzymes = [e[3:] for e in enzymes]
                    teste_dict['metadata'] = metadata
                    teste_dict['enzymes'] = enzymes
                    
                    with open(pilar_dump_path + '/{}'.format(filename), 'w') as f:
                        json.dump(teste_dict, f)
                

In [6]:
def get_taxon_values(raw_data, dump_path):
    taxa_dict = {}

    for i in raw_data:
        with open(i, 'r') as f:
            data = json.load(f)
            filename = '{}.json'.format(data['metadata']['Taxon ID'])
            metadata = data['metadata']
            if 'Lineage' not in metadata:
                pass
            elif 'Lineage' in metadata:
                lineage = metadata['Lineage']
                split_string = lineage.split(';')
                phylum = split_string[1]
                phylo_class = split_string[2]
                order = split_string[3]
                family = split_string[4]
                genus = split_string[5]
                species = split_string[6]
                taxon_id = data['metadata']['Taxon ID']

                taxa_dict[taxon_id] = [phylum, phylo_class, order, family, genus, species]
            
    with open(dump_path + '/genus_dict.json', 'w') as f:
        json.dump(taxa_dict, f)


In [127]:
get_taxon_values(eukarya_raw, eukarya_dump_path)
get_taxon_values(archaea_raw, archaea_dump_path)

In [128]:
get_taxon_values(bacteria_raw, bacteria_dump_path)

In [121]:
test_dict = {}
count = 0 
for i in bacteria_raw:
    with open(i, 'r') as f:
        test = json.load(f)
        metadata = test['metadata']
        if 'Lineage' not in metadata:
            pass
        elif 'Lineage' in metadata:
            lineage = metadata['Lineage']
            split_string = linage.split(';')
            phylum = split_string[1]
            phylo_class = split_string[2]
            order = split_string[3]
            family = split_string[4]
            genus = split_string[5]
            species = split_string[6]



Bacteria; Proteobacteria; Gammaproteobacteria; Pseudomonadales; unclassified; unclassified; Pseudomonadales bacterium GWC2_63_15
Bacteria; Actinobacteria; Actinobacteria; unclassified; unclassified; unclassified; unclassified
Bacteria; Actinobacteria; Actinobacteria; Micromonosporales; Micromonosporaceae; Micromonospora; Micromonospora echinospora
Bacteria; Proteobacteria; Alphaproteobacteria; Rhizobiales; Rhizobiaceae; Agrobacterium; Agrobacterium tumefaciens
Bacteria; Proteobacteria; unclassified; unclassified; unclassified; unclassified; unclassified
Bacteria; Proteobacteria; Betaproteobacteria; Burkholderiales; Burkholderiaceae; Polynucleobacter; Polynucleobacter meluiroseus
Bacteria; unclassified; unclassified; unclassified; unclassified; unclassified; candidate division WOR-1 bacterium RIFCSPHIGHO2_02_FULL_53_26
Bacteria; unclassified; unclassified; unclassified; unclassified; unclassified; uncultured bacterium
Bacteria; Candidatus Uhrbacteria; unclassified; unclassified; unclass

Bacteria; Proteobacteria; Betaproteobacteria; Burkholderiales; Comamonadaceae; Rhodoferax; Albidiferax sp. OV413
Bacteria; Chloroflexi; Dehalococcoidia; unclassified; unclassified; Dehalogenimonas; Dehalogenimonas lykanthroporepellens
Bacteria; Bacteroidetes; Bacteroidia; Bacteroidales; Prevotellaceae; Prevotella; Prevotella bryantii
Bacteria; Proteobacteria; Alphaproteobacteria; unclassified; unclassified; unclassified; Alphaproteobacteria bacterium JGI SC39-G12
Bacteria; Proteobacteria; Gammaproteobacteria; Pseudomonadales; Pseudomonadaceae; Pseudomonas; Pseudomonas sabulinigri
Bacteria; Firmicutes; unclassified; unclassified; unclassified; unclassified; unclassified
Bacteria; Proteobacteria; Betaproteobacteria; unclassified; unclassified; unclassified; Betaproteobacteria bacterium JGI M3C4D3-002-G15
Bacteria; Proteobacteria; Alphaproteobacteria; Rhizobiales; Rhizobiaceae; Rhizobium; Rhizobium sp. BK377
Bacteria; Actinobacteria; Actinobacteria; Streptomycetales; Streptomycetaceae; St

Bacteria; Proteobacteria; Hydrogenophilalia; Hydrogenophilales; unclassified; unclassified; Hydrogenophilales bacterium RIFOXYD1_FULL_62_11
Bacteria; Actinobacteria; Actinobacteria; Corynebacteriales; Mycobacteriaceae; Mycobacterium; Mycobacterium sp. URHD0025
Bacteria; Proteobacteria; Gammaproteobacteria; Pseudomonadales; Pseudomonadaceae; Pseudomonas; Pseudomonas salomonii
Bacteria; Proteobacteria; Alphaproteobacteria; Rhizobiales; Bradyrhizobiaceae; Bradyrhizobium; Bradyrhizobium shewense
Bacteria; Candidatus Magasanikbacteria; unclassified; unclassified; unclassified; unclassified; Candidatus Magasanikbacteria bacterium RIFCSPHIGHO2_02_FULL_47_14
Bacteria; Proteobacteria; Deltaproteobacteria; Desulfurellales; Desulfurellaceae; Hippea; Hippea maritima
Bacteria; Proteobacteria; Betaproteobacteria; Nitrosomonadales; unclassified; unclassified; Gallionellales bacterium GWA2_59_43
Bacteria; Bacteroidetes; unclassified; unclassified; unclassified; unclassified; Bacteroidetes bacterium RB

Bacteria; Proteobacteria; Alphaproteobacteria; Pelagibacterales; Pelagibacteraceae; Candidatus Pelagibacter; unclassified
Bacteria; unclassified; unclassified; unclassified; unclassified; unclassified; candidate division WOR-1 bacterium RIFOXYB2_FULL_45_9
Bacteria; Actinobacteria; Actinobacteria; Micrococcales; Microbacteriaceae; Glaciihabitans; Glaciihabitans tibetensis
Bacteria; Proteobacteria; Gammaproteobacteria; Vibrionales; Vibrionaceae; Vibrio; Vibrio cholerae
Bacteria; unclassified; unclassified; unclassified; unclassified; unclassified; unclassified
Bacteria; Candidatus Gottesmanbacteria; unclassified; unclassified; unclassified; unclassified; Candidatus Gottesmanbacteria bacterium RIFCSPLOWO2_01_FULL_46_9
Bacteria; Candidatus Giovannonibacteria; unclassified; unclassified; unclassified; unclassified; Candidatus Giovannonibacteria bacterium GW2011_GWB1_46_20
Bacteria; Proteobacteria; Gammaproteobacteria; Oceanospirillales; Oceanospirillaceae; Marinomonas; Marinomonas aquiplant

Bacteria; Proteobacteria; Gammaproteobacteria; Xanthomonadales; Xanthomonadaceae; Pseudoxanthomonas; Pseudoxanthomonas indica
Bacteria; Candidatus Daviesbacteria; unclassified; unclassified; unclassified; unclassified; Candidatus Daviesbacteria bacterium GW2011_GWA2_40_9
Bacteria; Bacteroidetes; unclassified; unclassified; unclassified; unclassified; unclassified
Bacteria; Proteobacteria; Alphaproteobacteria; Sphingomonadales; Sphingomonadaceae; Novosphingobium; Novosphingobium sp. BK486
Bacteria; Proteobacteria; Gammaproteobacteria; Xanthomonadales; Xanthomonadaceae; Xanthomonas; unclassified
Bacteria; Bacteroidetes; Bacteroidia; Bacteroidales; Porphyromonadaceae; Porphyromonas; Porphyromonas bennonis
Bacteria; Proteobacteria; Deltaproteobacteria; Myxococcales; Kofleriaceae; Haliangium; Haliangium ochraceum
Bacteria; Proteobacteria; Gammaproteobacteria; Pseudomonadales; Pseudomonadaceae; Pseudomonas; Pseudomonas corrugata
Bacteria; Candidatus Desantisbacteria; unclassified; unclassifi

...

Bacteria; candidate division WWE3; unclassified; unclassified; unclassified; unclassified; candidate division WWE3 bacterium CG08_land_8_20_14_0_20_43_13
Bacteria; Proteobacteria; Gammaproteobacteria; Pseudomonadales; Pseudomonadaceae; Pseudomonas; Pseudomonas sp. NFACC36
Bacteria; Candidatus Curtissbacteria; unclassified; unclassified; unclassified; unclassified; Candidatus Curtissbacteria bacterium GW2011_GWC1_44_33
Bacteria; Proteobacteria; Gammaproteobacteria; unclassified; unclassified; unclassified; gamma proteobacterium SCGC AB-629-P17
Bacteria; Proteobacteria; Alphaproteobacteria; Rhizobiales; Rhizobiaceae; Rhizobium; Rhizobium pusense
Bacteria; Actinobacteria; Actinobacteria; Micrococcales; Microbacteriaceae; Rhodoglobus; Rhodoglobus vestalii
Bacteria; Proteobacteria; Gammaproteobacteria; Xanthomonadales; Rhodanobacteraceae; Dyella; Dyella sp. SG609
Bacteria; Actinobacteria; Actinobacteria; Glycomycetales; Glycomycetaceae; Stackebrandtia; Stackebrandtia endophytica
Bacteria; A

Bacteria; Bacteroidetes; Flavobacteriia; Flavobacteriales; Flavobacteriaceae; Gramella; Gramella gaetbulicola
Bacteria; Firmicutes; Bacilli; Bacillales; Bacillaceae; Bacillus; Bacillus sp. cl96
Bacteria; Proteobacteria; Gammaproteobacteria; Vibrionales; Vibrionaceae; Vibrio; Vibrio sp. ES.044
Bacteria; Candidatus Uhrbacteria; unclassified; unclassified; unclassified; unclassified; Candidatus Uhrbacteria bacterium GW2011_GWF2_44_350
Bacteria; Bacteroidetes; Flavobacteriia; Flavobacteriales; Flavobacteriaceae; Zeaxanthinibacter; Zeaxanthinibacter enoshimensis
Bacteria; Proteobacteria; Betaproteobacteria; Burkholderiales; Burkholderiaceae; Burkholderia; Burkholderia sp. WSM2230
Bacteria; Cyanobacteria; unclassified; Synechococcales; Synechococcaceae; Synechococcus; Synechococcus sp. PCC 7336
Bacteria; Actinobacteria; Actinobacteria; Micrococcales; Microbacteriaceae; Agromyces; Agromyces sp. OV415
Bacteria; Verrucomicrobia; unclassified; unclassified; unclassified; unclassified; Verrucomic

Bacteria; Proteobacteria; Betaproteobacteria; Burkholderiales; Oxalobacteraceae; Duganella; Duganella sp. OV510
Bacteria; Firmicutes; Bacilli; Lactobacillales; Enterococcaceae; Tetragenococcus; Tetragenococcus muriaticus
Bacteria; Acidobacteria; unclassified; unclassified; unclassified; unclassified; Acidobacteria bacterium RIFCSPLOWO2_12_FULL_60_22
Bacteria; Bacteroidetes; Flavobacteriia; Flavobacteriales; Flavobacteriaceae; Olleya; Olleya sp. VCSM12
Bacteria; Proteobacteria; Alphaproteobacteria; Rhodospirillales; Rhodospirillaceae; Nitrospirillum; Nitrospirillum amazonense
Bacteria; Cyanobacteria; unclassified; Pleurocapsales; Hyellaceae; Pleurocapsa; Pleurocapsa sp. PCC 7319
Bacteria; Proteobacteria; Alphaproteobacteria; Rhodospirillales; Acetobacteraceae; Acidocella; Acidocella facilis
Bacteria; Actinobacteria; Actinobacteria; Micromonosporales; Micromonosporaceae; Micromonospora; Micromonospora olivasterospora
Bacteria; Actinobacteria; Actinobacteria; Streptomycetales; Streptomyce

Bacteria; Proteobacteria; Alphaproteobacteria; Rhizobiales; Rhizobiaceae; Agrobacterium; Agrobacterium sp. JGI 034
Bacteria; Firmicutes; Clostridia; Clostridiales; Clostridiales Family XIII. Incertae Sedis; Anaerovorax; Anaerovorax odorimutans
Bacteria; Bacteroidetes; unclassified; unclassified; unclassified; unclassified; Bacteroidetes bacterium RIFOXYC12_FULL_35_7
Bacteria; Proteobacteria; Zetaproteobacteria; Mariprofundales; Mariprofundaceae; Mariprofundus; Mariprofundus sp. JGI CrystG Aug08-3-L9
Bacteria; Candidatus Kaiserbacteria; unclassified; unclassified; unclassified; unclassified; Candidatus Kaiserbacteria bacterium RIFCSPHIGHO2_01_FULL_55_37
Bacteria; Bacteroidetes; Flavobacteriia; Flavobacteriales; Flavobacteriaceae; Maribacter; Maribacter polysiphoniae
Bacteria; Proteobacteria; Gammaproteobacteria; unclassified; unclassified; unclassified; uncultured gamma proteobacterium
Bacteria; Candidatus Woesebacteria; unclassified; unclassified; unclassified; unclassified; Candidatus

Bacteria; Candidatus Niyogibacteria; unclassified; unclassified; unclassified; unclassified; Candidatus Niyogibacteria bacterium RIFCSPLOWO2_01_FULL_45_48
Bacteria; Bacteroidetes; unclassified; unclassified; unclassified; unclassified; unclassified
Bacteria; Spirochaetes; unclassified; unclassified; unclassified; unclassified; unclassified
Bacteria; Bacteroidetes; Flavobacteriia; Flavobacteriales; Flavobacteriaceae; Flavobacterium; Flavobacterium haoranii
Bacteria; Tenericutes; Mollicutes; unclassified; unclassified; unclassified; Mollicutes bacterium JGI 000170CP_C05
Bacteria; Aquificae; Aquificae; Aquificales; Aquificaceae; Thermocrinis; Thermocrinis sp. GBS
Bacteria; Proteobacteria; Deltaproteobacteria; Desulfuromonadales; Geobacteraceae; Geobacter; Geobacter metallireducens
Bacteria; Proteobacteria; Alphaproteobacteria; unclassified; unclassified; unclassified; alpha proteobacterium SCGC AAA280-P20
Bacteria; Nitrospirae; unclassified; unclassified; unclassified; unclassified; Nitro

In [110]:
test_dict = {}
count = 0 
for i in archaea_raw:
    with open(i, 'r') as f:
        test = json.load(f)
        metadata = test['metadata']
        print(metadata['Lineage'])
            
        #lineage = metadata['Lineage']
        #split_string = lineage.split(';')
        #second = split_string[1]
        #print(second)
        
print(count)
print(len(bacteria_raw))


Archaea; Euryarchaeota; Thermoplasmata; Thermoplasmatales; unclassified; unclassified; Thermoplasmatales archaeon SCGC AB-539-C06
Archaea; Euryarchaeota; Halobacteria; Haloferacales; Haloferacaceae; Haloquadratum; Haloquadratum walsbyi
Archaea; unclassified; unclassified; unclassified; unclassified; unclassified; Candidatus Pacearchaeota archaeon
Archaea; Euryarchaeota; unclassified; unclassified; unclassified; unclassified; Euryarchaeota archaeon JGI CrystG Aug3-3-F14
Archaea; Crenarchaeota; Thermoprotei; Thermoproteales; Thermoproteaceae; Pyrobaculum; Pyrobaculum sp. WP30
Archaea; Euryarchaeota; Halobacteria; Natrialbales; Natrialbaceae; Haloterrigena; Haloterrigena saccharevitans
Archaea; Crenarchaeota; Thermoprotei; Thermoproteales; Thermoproteaceae; Thermoproteus; Thermoproteus uzoniensis
Archaea; Euryarchaeota; unclassified; unclassified; unclassified; unclassified; Euryarchaeota archaeon SM23-78
Archaea; Candidatus Geoarchaeota; unclassified; unclassified; unclassified; unclassi

Archaea; Crenarchaeota; unclassified; unclassified; unclassified; unclassified; unclassified
Archaea; Thaumarchaeota; unclassified; unclassified; unclassified; unclassified; unclassified
Archaea; Crenarchaeota; Thermoprotei; Sulfolobales; Sulfolobaceae; Sulfolobus; unclassified
Archaea; Euryarchaeota; Thermococci; Thermococcales; Thermococcaceae; Thermococcus; Thermococcus nautili
Archaea; Crenarchaeota; unclassified; unclassified; unclassified; unclassified; unclassified
Archaea; Crenarchaeota; Thermoprotei; Sulfolobales; Sulfolobaceae; Acidianus; Acidianus hospitalis
Archaea; Euryarchaeota; Archaeoglobi; Archaeoglobales; Archaeoglobaceae; Archaeoglobus; Archaeoglobus fulgidus
Archaea; Euryarchaeota; Methanomicrobia; Methanomicrobiales; Methanomicrobiaceae; Methanofollis; Methanofollis liminatans
Archaea; Crenarchaeota; Thermoprotei; Acidilobales; Acidilobaceae; Acidilobus; Acidilobus saccharovorans
Archaea; Euryarchaeota; Methanomicrobia; Methanosarcinales; Methanosarcinaceae; Methan

Archaea; Thaumarchaeota; unclassified; Nitrosopumilales; Nitrosopumilaceae; Nitrosopumilus; uncultured Candidatus Nitrosopumilus sp.
Archaea; Euryarchaeota; Candidatus Poseidoniia; Candidatus Poseidoniales; unclassified; unclassified; unclassified
Archaea; Euryarchaeota; Thermococci; Thermococcales; Thermococcaceae; Thermococcus; Thermococcus cleftensis
Archaea; Candidatus Bathyarchaeota; unclassified; unclassified; unclassified; unclassified; miscellaneous Crenarchaeota group archaeon SMTZ-80
Archaea; Euryarchaeota; Thermoplasmata; Thermoplasmatales; Thermoplasmataceae; Thermoplasma; Thermoplasma acidophilum
Archaea; Euryarchaeota; unclassified; unclassified; unclassified; unclassified; unclassified
Archaea; Thaumarchaeota; unclassified; unclassified; unclassified; unclassified; Thaumarchaeota archaeon
Archaea; unclassified; unclassified; unclassified; unclassified; unclassified; archaeon (Candidatus Huberarchaea) CG_4_9_14_0_8_um_filter_31_21
Archaea; Euryarchaeota; Methanobacteria; 

Archaea; Euryarchaeota; unclassified; unclassified; unclassified; unclassified; candidate division MSBL1 archaeon SCGC-AAA382A03
Archaea; Candidatus Aenigmarchaeota; unclassified; unclassified; unclassified; unclassified; Candidatus Aenigmarchaeota archaeon CG_4_8_14_3_um_filter_37_24
Archaea; Thaumarchaeota; unclassified; unclassified; unclassified; unclassified; unclassified
Archaea; Crenarchaeota; Thermoprotei; Thermoproteales; Thermoproteaceae; Vulcanisaeta; Vulcanisaeta sp. JCM 14467
Archaea; Euryarchaeota; Halobacteria; Haloferacales; Haloferacaceae; Haloferax; Haloferax sp. ATCC BAA-644
Archaea; Crenarchaeota; Thermoprotei; Sulfolobales; Sulfolobaceae; Metallosphaera; Metallosphaera sedula
Archaea; Euryarchaeota; unclassified; unclassified; unclassified; unclassified; candidate division MSBL1 archaeon SCGC-AAA261G05
Archaea; Euryarchaeota; Thermoplasmata; Thermoplasmatales; unclassified; unclassified; Thermoplasmatales archaeal group 2 archaeon JGI 01_L22
Archaea; Thaumarchaeota

Archaea; Euryarchaeota; Archaeoglobi; Archaeoglobales; Archaeoglobaceae; Archaeoglobus; Archaeoglobus sp. JdFR-26
Archaea; Crenarchaeota; Thermoprotei; Desulfurococcales; Desulfurococcaceae; Desulfurococcus; Desulfurococcus amylolyticus
Archaea; unclassified; unclassified; unclassified; unclassified; unclassified; archaeon
Archaea; Thaumarchaeota; unclassified; unclassified; unclassified; unclassified; Marine Group I thaumarchaeote SCGC AAA799-N04
Archaea; Euryarchaeota; Methanomicrobia; Methanosarcinales; Methanosarcinaceae; Methanosarcina; Methanosarcina mazei
Archaea; Euryarchaeota; Methanomicrobia; Methanosarcinales; Methanosarcinaceae; Methanolobus; Methanolobus profundi
Archaea; Thaumarchaeota; unclassified; Nitrosopumilales; Nitrosopumilaceae; Nitrosopumilus; unclassified
Archaea; Euryarchaeota; Halobacteria; Halobacteriales; Halobacteriaceae; unclassified; unclassified
Archaea; Euryarchaeota; Methanobacteria; Methanobacteriales; Methanobacteriaceae; Methanobacterium; Methanobac

Archaea; Crenarchaeota; Thermoprotei; Sulfolobales; Sulfolobaceae; Sulfolobus; unclassified
Archaea; Candidatus Heimdallarchaeota; unclassified; unclassified; unclassified; unclassified; Candidatus Heimdallarchaeota archaeon LC_2
Archaea; Aenigmarchaeota; unclassified; unclassified; unclassified; Candidatus Aenigmarchaeum; subterraneum
Archaea; Euryarchaeota; unclassified; unclassified; unclassified; unclassified; Euryarchaeota archaeon SCGC AD-261-K09
Archaea; Crenarchaeota; Thermoprotei; Sulfolobales; Sulfolobaceae; Sulfolobus; Sulfolobus acidocaldarius
Archaea; Crenarchaeota; Thermoprotei; Sulfolobales; Sulfolobaceae; Sulfolobus; Sulfolobus islandicus
Archaea; Euryarchaeota; Halobacteria; Natrialbales; Natrialbaceae; Natrialba; Natrialba magadii
Archaea; Euryarchaeota; Methanomicrobia; Methanosarcinales; Methanosarcinaceae; Methanosarcina; Methanosarcina barkeri
Archaea; Crenarchaeota; Thermoprotei; Desulfurococcales; unclassified; unclassified; unclassified
Archaea; Crenarchaeota; 

Archaea; Crenarchaeota; Thermoprotei; Sulfolobales; Sulfolobaceae; Sulfolobus; Sulfolobus acidocaldarius
Archaea; Euryarchaeota; unclassified; unclassified; unclassified; unclassified; Euryarchaeota archaeon
Archaea; unclassified; unclassified; unclassified; unclassified; unclassified; Candidatus Pacearchaeota archaeon RBG_16_35_8
Archaea; unclassified; unclassified; unclassified; unclassified; unclassified; unclassified
Archaea; Euryarchaeota; Methanomicrobia; Methanosarcinales; Methanosarcinaceae; Methanosarcina; Methanosarcina mazei
Archaea; Euryarchaeota; Halobacteria; Haloferacales; Haloferacaceae; Halopelagius; Halopelagius longus
Archaea; Euryarchaeota; Methanobacteria; Methanobacteriales; Methanobacteriaceae; Methanobacterium; Methanobacterium formicicum
Archaea; Euryarchaeota; Methanomicrobia; Methanosarcinales; Methanosarcinaceae; Methanosarcina; Methanosarcina mazei
Archaea; Candidatus Parvarchaeota; unclassified; unclassified; unclassified; unclassified; unclassified
Archae

Archaea; Euryarchaeota; Halobacteria; Halobacteriales; unclassified; unclassified; haloarchaeon 3A1_DGR
Archaea; Thaumarchaeota; unclassified; unclassified; unclassified; unclassified; Marine Group I thaumarchaeote SCGC AB-629-I23
Archaea; Euryarchaeota; Halobacteria; Natrialbales; Natrialbaceae; Natrialba; Natrialba magadii
Archaea; Euryarchaeota; Methanomicrobia; Methanomicrobiales; Methanoregulaceae; unclassified; unclassified
Archaea; Euryarchaeota; Halobacteria; unclassified; unclassified; unclassified; unclassified
Archaea; Euryarchaeota; Thermococci; Thermococcales; Thermococcaceae; Pyrococcus; Pyrococcus sp. ST04
Archaea; Thaumarchaeota; unclassified; unclassified; unclassified; unclassified; Thaumarchaeota archaeon SCGC AC-312_F15
Archaea; Crenarchaeota; Thermoprotei; Sulfolobales; Sulfolobaceae; Metallosphaera; Metallosphaera sedula
Archaea; Euryarchaeota; Thermococci; Thermococcales; Thermococcaceae; Thermococcus; Thermococcus sp. 2319x1
Archaea; Candidatus Diapherotrites; u

Archaea; Euryarchaeota; Halobacteria; Halobacteriales; Halobacteriaceae; Halobacterium; Halobacterium hubeiense
Archaea; Thaumarchaeota; unclassified; Nitrosopumilales; unclassified; unclassified; unclassified
Archaea; Euryarchaeota; Methanomicrobia; Methanosarcinales; unclassified; unclassified; Methanosarcinales archaeon Methan_04
Archaea; Geoarchaeota; unclassified; unclassified; unclassified; unclassified; unclassified
Archaea; Crenarchaeota; Thermoprotei; Sulfolobales; Sulfolobaceae; Sulfolobus; Sulfolobus islandicus
Archaea; Euryarchaeota; Methanomicrobia; Methanomicrobiales; Methanomicrobiaceae; Methanolacinia; Methanolacinia petrolearia
Archaea; unclassified; unclassified; unclassified; unclassified; unclassified; Candidatus Pacearchaeota archaeon
Archaea; Crenarchaeota; Thermoprotei; Sulfolobales; Sulfolobaceae; Sulfolobus; Sulfolobus islandicus
Archaea; Crenarchaeota; Thermoprotei; Sulfolobales; Sulfolobaceae; Sulfolobus; Sulfolobus islandicus
Archaea; Euryarchaeota; Methanom

Archaea; Euryarchaeota; Methanobacteria; Methanobacteriales; Methanobacteriaceae; Methanobacterium; Methanobacterium formicicum
Archaea; Euryarchaeota; Halobacteria; Halobacteriales; Halobacteriaceae; unclassified; unclassified
Archaea; Euryarchaeota; Archaeoglobi; Archaeoglobales; Archaeoglobaceae; Geoglobus; Geoglobus acetivorans
Archaea; unclassified; unclassified; unclassified; unclassified; unclassified; unclassified
Archaea; Euryarchaeota; unclassified; unclassified; unclassified; unclassified; unclassified
Archaea; unclassified; unclassified; unclassified; unclassified; unclassified; unclassified
Archaea; Euryarchaeota; Halobacteria; Natrialbales; Natrialbaceae; Haloterrigena; Haloterrigena salina
Archaea; Candidatus Bathyarchaeota; unclassified; unclassified; unclassified; unclassified; miscellaneous Crenarchaeota group-1 archaeon SG8-32-1
Archaea; Candidatus Micrarchaeota; unclassified; unclassified; unclassified; unclassified; Candidatus Micrarchaeota archaeon CG06_land_8_20_

In [8]:
create_eclists_ecdict_genome(eukarya_raw, eukarya_dump_path)
create_eclists_ecdict_genome(archaea_raw, archaea_dump_path)
create_eclists_ecdict_metagenome(metagenomes_raw, metagenome_dump_path)
create_eclists_ecdict_genome(bacteria_raw, bacteria_dump_path)